In [2]:
import requests
import sys, os
sys.path.append('../capture_dou')

In [3]:
import cloudscraper
import parse_dou_article as pa

In [4]:

# The lists inside relevant_articles will receive the articles selected by each filter set:
url_file_list = ['https://www.in.gov.br/web/dou/-/portaria-n-256-de-13-de-dezembro-de-2022-457714216']
relevant_articles = [[]]*len(url_file_list)
     

# Using cloudscraper as a workaround for the CloudFlare protection:
scraper = cloudscraper.create_scraper(delay=10,   browser={'custom': 'ScraperBot/1.0',})
response = scraper.get(url_file_list[0])


raw_article = pa.parse_dou_article(response, url_file_list[0])


## Replicando parsing

In [8]:
article = pa.select_article(response)
data = pa.recurse_over_nodes(article, None, {})

In [9]:
data

{'row-fluid': '\n | \n | \n\n | \n | \n | \n',
 'row-fluid_cabecalho-dou-text-center': '\n | \n | \n',
 'row-fluid_cabecalho-dou-text-center_brasao-brasil': '\n | \n',
 'row-fluid_cabecalho-dou-text-center_brasao-brasil_d-none': 'BrasÃ£o do Brasil',
 'row-fluid_cabecalho-dou-text-center_cabecalho-titulo-dou': 'DiÃ¡rio Oficial da UniÃ£o',
 'row-fluid_detalhes-dou': '\n | \n | \n',
 'row-fluid_detalhes-dou_text-center': '\n | \n | \n | \n | \n | \n | \n | \n | \n | \n | \n | \n | \n | \n',
 'row-fluid_detalhes-dou_text-center_publicado-dou': 'Publicado em: ',
 'row-fluid_detalhes-dou_text-center_publicado-dou-data': '13/01/2023',
 'row-fluid_detalhes-dou_text-center_pipe': ' |  |  |  |  | ',
 'row-fluid_detalhes-dou_text-center_edicao-dou': 'EdiÃ§Ã£o: ',
 'row-fluid_detalhes-dou_text-center_edicao-dou-data': '10',
 'row-fluid_detalhes-dou_text-center_secao-dou': 'SeÃ§Ã£o: 2 | PÃ¡gina: ',
 'row-fluid_detalhes-dou_text-center_secao-dou-data': '46',
 'row-fluid_detalhes-dou_text-center_orga

In [14]:
raw_article

[{'key': 'd-none',
  'value': 'Brasão do Brasil | Borda do rodapé | Logo da Imprensa',
  'url': 'https://www.in.gov.br/web/dou/-/portaria-n-256-de-13-de-dezembro-de-2022-457714216',
  'capture_date': '2023-01-13 14:06:18',
  'url_certificado': 'http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=13/01/2023&jornal=529&pagina=46'},
 {'key': 'cabecalho-titulo-dou',
  'value': 'Diário Oficial da União',
  'url': 'https://www.in.gov.br/web/dou/-/portaria-n-256-de-13-de-dezembro-de-2022-457714216',
  'capture_date': '2023-01-13 14:06:18',
  'url_certificado': 'http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=13/01/2023&jornal=529&pagina=46'},
 {'key': 'publicado-dou',
  'value': 'Publicado em: ',
  'url': 'https://www.in.gov.br/web/dou/-/portaria-n-256-de-13-de-dezembro-de-2022-457714216',
  'capture_date': '2023-01-13 14:06:18',
  'url_certificado': 'http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=13/01/2023&jornal=529&pagina=46'},
 {'key': 'public

## Raw Code

In [ ]:

            # Organize article by capturing selected fields:
            if gs.debug:
                print("Select relevant fields...")        
            article = sa.structure_article(raw_article)
            
            # Write raw article's file to database:
            wrote_return = 2   # (Preset status of 'save article' operation)
            if config['save_articles']:
                if gs.debug:
                    print("Saving article...")
                if gs.local:
                    wa.write_local_article(config, raw_article, url_file['filename'])
                    wrote_return = 200
                else:
                    write_return = wa.write_to_s3(config, raw_article, url_file['filename'])
                    if write_return == 200:
                        wrote_return = wa.copy_s3_to_storage_gcp(config['bucket'], config['key'] + url_file['filename'])
                        if wrote_return != 200 and ds.debug:
                            print('Copy_s3_to_storage_gcp failed.') 
                    elif ds.debug:
                        print('Write_to_s3 failed.')
                        
            # Loop over filters:
            if gs.debug:
                print("Filtering article...")
            for i in range(len(bot_infos)):
                # Filter article:
                relevant_articles[i] = relevant_articles[i] + fa.get_relevant_articles(bot_infos[i], [article])
                # Slack crashes if message has more than 50 blocks.
                # Avoid this by pre-posting long messages:
                if config['post_articles'] and len(relevant_articles[i]) > 20:
                    if gs.debug:
                        print('Selected more than 20 articles.')
                    ps.post_article(config, bot_infos[i], relevant_articles[i])
                    relevant_articles[i] = []

            # Record URL in list of captured articles (for now, we will assume that the article always was posted):
            if captured_article_ok(config['save_articles'], wrote_return==200, config['post_articles'], True):
                gu.register_captured_url(config['url_list'], url_file['url'])
            elif gs.debug:
                print('Failed to record as done: ' + url_file['url'])
                
        else:
            # GET ran but returned BAD STATUS:
            print('Bad status in GET ' + url_file['url'])  
# End of Loop over URLs.

if config['post_articles']:
    # Send the selected articles to Slack:
    for i in range(len(bot_infos)):
        if len(relevant_articles[i]) > 0:
            ps.post_article(config, bot_infos[i], relevant_articles[i])

# Return the config for next capture try:
return next_config
